Notebook creator: [Nikolaos Tsopanidis](https://github.com/tSopermon)
# Summarization App Creation Notebook
Material aqcuired from [Chanin Nantasenamat](https://blog.streamlit.io/langchain-tutorial-3-build-a-text-summarization-app/)

## Text Summarization
Text summarization uses natural language processing to condense long-form content into a shorter summary while retaining key points. 

There are two types:
 1. **Extractive summarization**: Selects and combines key phrases or sentences from the original text without altering them.
 2. **Abstractive summarization**: Understands the main ideas and generates a concise, paraphrased summary, requiring deeper comprehension.

## App overview
 1. **Input text submission**: to be summarized into the Streamlit frontend.
 2. **Text preprocessing**: text is splitted into smaller chunks, creating documents for each chunk. This will help LLM model to summarize faster.
 3. **Summary display**: how the summary will be displayed in the app.

## Development process
 * **Input** text will be splitted into chunks using the `CharacterTextSplitter()` function from LangChain framework and we will create documents for each chunk using `Document()` function.
 * The LLM model will generate an **Output** from the processed input, using the `load_summarize_chain()` function.

Steps are displayed below:

### **Step 1**: Setting up the code environment

In [ ]:
#%pip install streamlit langchain langchain-huggingface tiktoken

### **Step 2**: Importing necessary libraries

In [ ]:
import streamlit as st
from langchain_community.llms import OpenAI
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

### **Step 3**: Building the app

In [ ]:
def generate_response(txt, openai_api_key):
    llm = OpenAI(
        temperature=0,
        openai_api_key= openai_api_key
    )
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=20,
        length_function=len
    )
    texts = text_splitter.split_text(txt)
    # Create multiple documents
    docs = [Document(page_content=t) for t in texts]
    # summarization chain
    chain = load_summarize_chain(
        llm=llm,
        chain_type="map_reduce"
    )
    return chain.invoke(docs)

In [ ]:
# Page title
st.set_page_config(page_title="Text Submission with Hugging Face and LangChain", page_icon=":guardsman:")
st.title("Text Submission with Hugging Face and LangChain")

# Text input
txt_input = st.text_area("Enter your text:", height=200)

# form to accept user input
result = []
with st.form('summarize_form', clear_on_submit=True):
    openai_api_key = st.text_input('OpenAI API Key', type = 'password', disabled=not txt_input)
    submitted = st.form_submit_button("Submit")
    if submitted and openai_api_key.startswith("sk-"):
        with st.spinner("Generating summary..."):
            # Call the function to generate response
            response = generate_response(txt_input, openai_api_key)
            result.append(response)
            del openai_api_key
# Display the result
if len(result):
    st.info(response)

### To use, create an OpenAI key from this [link](https://platform.openai.com/api-keys) to insert it on the app